<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Entity-Resolution-Practice:-Regulation-Data" data-toc-modified-id="Entity-Resolution-Practice:-Regulation-Data-1">Entity Resolution Practice: Regulation Data</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1">Introduction</a></span></li><li><span><a href="#Make-A-Practice-Dataset" data-toc-modified-id="Make-A-Practice-Dataset-1.2">Make A Practice Dataset</a></span></li><li><span><a href="#Practice" data-toc-modified-id="Practice-1.3">Practice</a></span></li><li><span><a href="#Discussion" data-toc-modified-id="Discussion-1.4">Discussion</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-1.5">Conclusion</a></span></li></ul></li></ul></div>

# Entity Resolution Practice: Regulation Data

## Introduction

This practice is designed to realize entity resolution on a real dataset. As rare real datasets can be used in practice, a practice data will be generated from a published data on [datashanghai](http://www.datashanghai.gov.cn/) and mixed with randomly sampled rows from itself. 

The most important part of the project lays in the **Practice**, which will introduce `recordlinkage` module and apply it to our dataset `regulation_dup`.

## Make A Practice Dataset

The data contains regulation information of company in Shanghai, downloaded from [datashanghai](http://www.datashanghai.gov.cn/). The made data contains 5% duplicated companies but each row has a unique index. 

In [18]:
import os
import pandas as pd

wd = '/Users/ewenwang/Documents/practice_data'
file = 'regulation.csv'

os.chdir(wd)
regulation = pd.read_csv(file)

In [13]:
regulation.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [14]:
regulation.shape

(36672, 7)

In [19]:
seed = 2017
n_dup = round(regulation.shape[0]*.05)
duplication = regulation.sample(n=n_dup, replace=True, random_state=seed)

In [56]:
duplication.shape

(1834, 7)

In [57]:
duplication.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
16186,16187,上海嘉方市政建筑有限公司,631661753,上海市嘉定区安亭镇翔方公路２３７７号,嘉定,5/31/16 2:05,未发现问题。
8963,8964,上海尚原餐饮管理有限公司,692932609,上海市浦东新区苗圃路６３号１－３层,上海市浦东新区苗圃路６３号１－３层,11/20/15 10:11,未发现问题。
7727,7728,上海万辉汽车附件有限公司,630910922,上海市浦东新区万祥镇万灵路１３８号,南汇万祥工业区,9/24/15 9:09,未发现问题。
1224,1225,上海申得欧有限公司,607299571,上海市浦东新区庆达路２８８号,庆达路288号,3/12/14 3:03,未发现问题。 其他问题：叉车未张贴有效期内的检验合格标志
30208,30209,上海顺城物流有限公司,07483557X,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,12/5/16 11:12,在检查“场（厂）内机动车辆使用情况(叉车，5110310112201512003 7)”时发...


In [20]:
duplication['序号'] = range(regulation.shape[0]+1, regulation.shape[0]+1+duplication.shape[0])
duplication.index = range(regulation.shape[0]+1, regulation.shape[0]+1+duplication.shape[0])

In [61]:
duplication.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
36673,36673,上海嘉方市政建筑有限公司,631661753,上海市嘉定区安亭镇翔方公路２３７７号,嘉定,5/31/16 2:05,未发现问题。
36674,36674,上海尚原餐饮管理有限公司,692932609,上海市浦东新区苗圃路６３号１－３层,上海市浦东新区苗圃路６３号１－３层,11/20/15 10:11,未发现问题。
36675,36675,上海万辉汽车附件有限公司,630910922,上海市浦东新区万祥镇万灵路１３８号,南汇万祥工业区,9/24/15 9:09,未发现问题。
36676,36676,上海申得欧有限公司,607299571,上海市浦东新区庆达路２８８号,庆达路288号,3/12/14 3:03,未发现问题。 其他问题：叉车未张贴有效期内的检验合格标志
36677,36677,上海顺城物流有限公司,07483557X,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,上海市闵行区莲花南路２５８８号５幢二层Ｂ区,12/5/16 11:12,在检查“场（厂）内机动车辆使用情况(叉车，5110310112201512003 7)”时发...


In [68]:
duplication.tail()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
38502,38502,上海沪菲电缆有限公司,774778246,上海市松江区中山街道荣乐东路１８号Ａ区,上海市金山区朱泾镇温河村新民1109号,5/31/16 2:05,在检查“场（厂）内机动车辆使用情况(叉车，5110310116201205002 2)”时发...
38503,38503,上海乐益物业有限公司,134501417,上海市普育东路１８５号１０２室,重庆北路318号,4/18/16 2:04,设备登记使用单位为上海捷艾尔物业管理有限公司。移动端登记为上海乐益物业有限公司。
38504,38504,上海凯顿酒店管理有限公司,776252678,上海市浦东唐镇创新中路６０１号８幢２０１室,上海市浦东唐镇创新中路６０１号８幢２０１室,11/2/15 3:11,未发现问题。
38505,38505,上海凡宏实业有限公司,630977326,上海市松江区洞泾镇洞舟路６１９号,上海市松江区洞泾镇洞舟路６１９号,7/8/16 12:07,未发现问题。
38506,38506,上海富达日用化工厂,134423527,上海市崇明县北沿公路１２１８号,上海市崇明县前进农场,12/27/13 10:12,未发现问题。


In [21]:
regulation_dup = pd.concat([regulation, duplication])
regulation_dup.shape

(38506, 7)

In [22]:
regulation_dup.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [23]:
regulation_dup.tail()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
38502,38502,上海沪菲电缆有限公司,774778246,上海市松江区中山街道荣乐东路１８号Ａ区,上海市金山区朱泾镇温河村新民1109号,5/31/16 2:05,在检查“场（厂）内机动车辆使用情况(叉车，5110310116201205002 2)”时发...
38503,38503,上海乐益物业有限公司,134501417,上海市普育东路１８５号１０２室,重庆北路318号,4/18/16 2:04,设备登记使用单位为上海捷艾尔物业管理有限公司。移动端登记为上海乐益物业有限公司。
38504,38504,上海凯顿酒店管理有限公司,776252678,上海市浦东唐镇创新中路６０１号８幢２０１室,上海市浦东唐镇创新中路６０１号８幢２０１室,11/2/15 3:11,未发现问题。
38505,38505,上海凡宏实业有限公司,630977326,上海市松江区洞泾镇洞舟路６１９号,上海市松江区洞泾镇洞舟路６１９号,7/8/16 12:07,未发现问题。
38506,38506,上海富达日用化工厂,134423527,上海市崇明县北沿公路１２１８号,上海市崇明县前进农场,12/27/13 10:12,未发现问题。


In [69]:
regulation_dup.to_excel('regulation_dup.xlsx', index = False)

## Practice

Task: Try to find out all indexes that belong to each company via its registered address and production address.

In [1]:
import os
import pandas as pd

wd = '/Users/ewenwang/Documents/practice_data'
file = 'regulation_dup.csv'

os.chdir(wd)
regulation_dup = pd.read_csv(file)

In [2]:
sample = regulation_dup.head(10)
sample.head()

,序号,企业名称,组织机构代码,注册地址,生产地址,检查日期,处理措施
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [3]:
sample.columns = ['index', 'name', 'company_id', 'registered_add', 'production_add', 'date', 'content']

In [4]:
sample.head()

,index,name,company_id,registered_add,production_add,date,content
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [5]:
regulation_dup.columns = ['index', 'name', 'company_id', 'registered_add', 'production_add', 'date', 'content']

In [6]:
regulation_dup.head()

,index,name,company_id,registered_add,production_add,date,content
0,1,上海巴士五汽公共交通有限公司,607392127,上海市共和新路３０３４号,南陈路98号,1/8/10 1:01,未发现问题。
1,2,上海康志物业管理有限公司,86158393,上海市宝山区锦秋路４８号Ｇ１５,上海市宝山区锦秋路４８号Ｇ１５,1/8/10 8:01,未发现问题。
2,3,上海地矿物业管理有限公司,630350240,上海市沪太路１０６３弄５号甲,灵石路930号,1/8/10 9:01,未发现问题。
3,4,上海立新五金厂,132808298,上海市静安区南京西路１９５５弄９号,南陈路24号,1/8/10 9:01,未发现问题。
4,5,上海同祁置业有限公司,586791275,上海市宝山区锦秋路４８号Ｆ１３８,上海市宝山区锦秋路４８号Ｆ１３８,1/8/10 9:01,未发现问题。


In [7]:
import recordlinkage

indexer = recordlinkage.FullIndex()

In [8]:
pairs = indexer.index(sample)
indexer = recordlinkage.BlockIndex(on='registered_add')
pairs = indexer.index(sample)

print(len(pairs))

0


In [9]:
type(sample)

pandas.core.frame.DataFrame

In [10]:
type(regulation_dup)

pandas.core.frame.DataFrame

In [11]:
pairs = indexer.index(regulation_dup)
indexer = recordlinkage.BlockIndex(on='registered_add')
pairs = indexer.index(regulation_dup)

print(len(pairs))

56051


In [12]:
compare_cl = recordlinkage.Compare()

compare_cl.string('registered_add', 'registered_add', method='jarowinkler', threshold=0.85, label='registered_add')
compare_cl.string('production_add', 'production_add', method='jarowinkler', threshold=0.85, label='production_add')

features = compare_cl.compute(pairs, regulation_dup)

In [13]:
features.head(10)

registered_add  production_add
2     21530             1.0             1.0
3     23032             1.0             1.0
4     20566             1.0             1.0
      27518             1.0             1.0
      36223             1.0             1.0
20566 27518             1.0             1.0
      36223             1.0             1.0
27518 36223             1.0             1.0
5     21989             1.0             1.0
6     33683             1.0             1.0

In [14]:
# features.index.values
pairs.values[0:10]

array([(2, 21530), (3, 23032), (4, 20566), (4, 27518), (4, 36223),
       (20566, 27518), (20566, 36223), (27518, 36223), (5, 21989),
       (6, 33683)], dtype=object)

In [15]:
pairs.values[0]

(2, 21530)

In [16]:
print(pairs.values[0], 
      pairs.values[0][0], regulation_dup.values[pairs.values[0][0],1], 
      pairs.values[0][1], regulation_dup.values[pairs.values[0][1],1])

(2, 21530) 2 上海地矿物业管理有限公司 21530 上海地矿物业管理有限公司


In [17]:
def print_pairs(pairs, n=10):
    for i in range(n):
        print(pairs.values[i], 
              pairs.values[i][0], regulation_dup.values[pairs.values[i][0],1], 
              pairs.values[i][1], regulation_dup.values[pairs.values[i][1],1])
    return None

In [18]:
print_pairs(pairs, 10)

(2, 21530) 2 上海地矿物业管理有限公司 21530 上海地矿物业管理有限公司
(3, 23032) 3 上海立新五金厂 23032 上海立新五金厂
(4, 20566) 4 上海同祁置业有限公司 20566 上海同祁置业有限公司
(4, 27518) 4 上海同祁置业有限公司 27518 上海同祁置业有限公司
(4, 36223) 4 上海同祁置业有限公司 36223 上海同祁置业有限公司
(20566, 27518) 20566 上海同祁置业有限公司 27518 上海同祁置业有限公司
(20566, 36223) 20566 上海同祁置业有限公司 36223 上海同祁置业有限公司
(27518, 36223) 27518 上海同祁置业有限公司 36223 上海同祁置业有限公司
(5, 21989) 5 上海高地物业管理有限公司虹口分公司 21989 上海高地物业管理有限公司虹口分公司
(6, 33683) 6 上海康通电缆组件有限公司 33683 上海康通电缆组件有限公司


In [19]:
regulation_dup.values[pairs.values[0][0],1] == regulation_dup.values[pairs.values[0][1],1]

True

In [20]:
len(pairs.values)

56051

In [21]:
regulation_dup['name'][0]

'上海巴士五汽公共交通有限公司'

In [43]:
import numpy as np

results = pd.DataFrame(columns=['pairs', 'first', 'second', 'result'], index=np.arange(len(pairs.values)))

for index, value in enumerate(pairs.values):
    results['pairs'][index] = value
    results['first'][index] = regulation_dup.values[value[0],1]
    results['second'][index] = regulation_dup.values[value[1],1]
    results['result'][index] = 1 if regulation_dup.values[value[0],2] == regulation_dup.values[value[1],2] else 0

In [48]:
accuracy = results['result'].values.sum()/results.shape[0]
accuracy

0.9491534495370287

In [55]:
seed = 2018
results.sample(10, random_state=seed)

,pairs,first,second,result
48789,"(21161, 34172)",上海天伦医院有限公司,上海天伦医院有限公司,1
31673,"(3136, 4580)",上海万宇房地产（集团）有限公司,上海万宇房地产（集团）有限公司,1
38667,"(5867, 10381)",上海金玉兰物业管理有限公司,上海金玉兰物业管理有限公司,1
19741,"(10355, 31254)",上海久创建设管理有限公司,上海磁浮交通发展有限公司,0
37881,"(7080, 34605)",上海慧山投资管理有限公司,上海慧山投资管理有限公司,1
14237,"(9198, 9214)",上海中医药大学附属曙光医院,上海中医药大学附属曙光医院,1
48951,"(14303, 14702)",上海利通置业有限公司,上海利通置业有限公司,1
51608,"(16872, 16876)",上海光明荷斯坦牧业有限公司,上海光明荷斯坦牧业有限公司,1
18844,"(22504, 36895)",上海地铁运营有限公司客运一分公司,上海地铁运营有限公司客运一分公司,1
50750,"(15573, 22509)",上海岩森实业发展有限公司,上海岩森实业发展有限公司,1


In [77]:
wrong = results.iloc[np.where(results.iloc[:, 3] == 0)[0],:]

In [78]:
wrong.shape

(2850, 4)

In [79]:
wrong.head()

,pairs,first,second,result
51,"(11, 10854)",新世界百货（中国）有限公司,上海中燃船舶燃料有限公司,0
52,"(11, 11223)",新世界百货（中国）有限公司,上海福德物业管理经营有限公司,0
53,"(11, 16032)",新世界百货（中国）有限公司,上海福德物业管理经营有限公司,0
55,"(11, 24369)",新世界百货（中国）有限公司,上海福德物业管理经营有限公司,0
56,"(11, 35551)",新世界百货（中国）有限公司,上海福德物业管理经营有限公司,0


In [83]:
print(regulation_dup['registered_add'][11], '\t',
      regulation_dup['registered_add'][10854], '\t',
      regulation_dup['registered_add'][11223])
print()
print(regulation_dup['production_add'][11], '\t\t',
      regulation_dup['production_add'][10854], '\t',
      regulation_dup['production_add'][11223])

上海市四川北路１６８８号 	 上海市四川北路１６８８号 	 上海市四川北路１６８８号

四川北路1688号 		 杨浦区军工路3500号 	 上海市四川北路１６８８号


In [46]:
results.to_excel('regulation_results.xlsx', index=False)

Features Analysis

In [85]:
features.describe()

,registered_add,production_add
count,56051.0,56051.000000
mean,1.0,0.960143
std,0.0,0.195624
min,1.0,0.000000
25%,1.0,1.000000
50%,1.0,1.000000
75%,1.0,1.000000
max,1.0,1.000000


In [86]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0    53817
1.0     2234
dtype: int64

In [88]:
matches = features[features.sum(axis=1) > 0]

print(len(matches))
matches.head(10)

56051


registered_add  production_add
2     21530             1.0             1.0
3     23032             1.0             1.0
4     20566             1.0             1.0
      27518             1.0             1.0
      36223             1.0             1.0
20566 27518             1.0             1.0
      36223             1.0             1.0
27518 36223             1.0             1.0
5     21989             1.0             1.0
6     33683             1.0             1.0

## Discussion

In the practice above, we didn't consider Chinese characters as an issue, which potentially be a problem in reality. As a result, we introduce a translation module `googletrans` as following, which may be used in the future but not included in this project.

In [86]:
from googletrans import Translator

translator = Translator()

example = translator.translate('안녕하세요.')
print(example.text)

translator.translate('科苑路').text

Good morning.


'Keyuan Road'

## Conclusion

In this project, we made a dataset to practice data deduplication or entity resolution, which is to find records in datasets belonging to the same entity. In detail, we employed `recordlinkage` module to realize our goal. 

**Steps of Data Deduplication:**

1. Make record pairs. 
2. Use **block** to fix identity columns and control the number of records to be passed to next step.
3. Compare records on specific columns, which can be regarded as **a weak block**.
4. *Manually double check.* 

**Note:** The step 4 is not included in our project but can be employed in practice.

In the discussion part, we introduced a potential issue of Chinese characters, which may be considered in practice as well.